# 📖 Markdown 清洗脚本

将OCR输出的原始Markdown按照章节结构重新组织，同时保留页码信息用于RAG/知识库建立

In [1]:
import os
import re
from pathlib import Path
from typing import List, Dict, Tuple, Optional
# 配置路径
input_dir = Path('output_markdown')
output_dir = Path('cleaned_md')
output_dir.mkdir(exist_ok=True)

print(f"输入目录: {input_dir}")
print(f"输出目录: {output_dir}")
print(f"创建输出目录: {output_dir}")

输入目录: output_markdown
输出目录: cleaned_md
创建输出目录: cleaned_md


In [2]:
import re
import unicodedata
from pathlib import Path
from typing import Dict, List, Tuple, Optional


class MarkdownCleaner:
    """
    针对 EPUB -> pandoc -> Markdown 场景的清洗器（兼容你原来 OCR 清洗思路）
    目标：
    1) 清理 pandoc/calibre 残留 HTML 与锚点噪音
    2) 统一 superscript 引用（如 <sup>...</sup>）为 [^n]
    3) 修复被错误断开的行（避免把标题/列表/代码误并）
    4) 保留 Markdown 标题结构，便于后续 AI 识别
    """

    def __init__(
        self,
        input_path: str,
        output_path: Optional[str] = None,
        remove_title_lines: bool = True,
        convert_sup_to_footnote: bool = True,  # True => [^n], False => [n]
    ):
        self.input_path = Path(input_path)
        if output_path is None:
            self.output_path = self.input_path.with_name(self.input_path.stem + "_cleaned.md")
        else:
            self.output_path = Path(output_path)

        self.remove_title_lines = remove_title_lines
        self.convert_sup_to_footnote = convert_sup_to_footnote

        # 统计信息，便于你做质量回归
        self.stats: Dict[str, int] = {
            "html_tags_removed": 0,
            "sup_refs_normalized": 0,
            "index_anchors_removed": 0,
            "escaped_brackets_fixed": 0,
            "lines_merged": 0,
            "book_title_lines_removed": 0,
        }

        # 你原 notebook 若有固定书名列表，可在这里补充
        self.book_title_patterns = [
            r"^\s*《[^》]+》\s*$",
            r"^\s*[\u4e00-\u9fffA-Za-z0-9·\s]{2,60}\s*[-—]\s*[\u4e00-\u9fffA-Za-z0-9·\s]{2,60}\s*$",
        ]

    # -----------------------------
    # 基础工具
    # -----------------------------
    @staticmethod
    def _normalize_unicode(text: str) -> str:
        # 统一全半角兼容字符，减少奇怪符号
        return unicodedata.normalize("NFKC", text)

    @staticmethod
    def _collapse_spaces(text: str) -> str:
        text = re.sub(r"[ \t]+", " ", text)
        text = re.sub(r"\n{3,}", "\n\n", text)
        return text.strip()

    # -----------------------------
    # 1) pandoc 引用噪音清洗
    # -----------------------------
    def normalize_pandoc_refs(self, text: str) -> str:
        """
        处理：
        - <sup>...</sup> 内部的各种 span/link/转义符
        - (#index_split_xxx.html_fileposxxxxx)
        - \\[7\\] -> [7]
        """
        s = text

        # HTML 实体
        s = s.replace("&nbsp;", " ").replace("&amp;", "&")

        # 先替换转义方括号（避免后续匹配不到）
        before = s
        s = s.replace(r"\[", "[").replace(r"\]", "]")
        if s != before:
            self.stats["escaped_brackets_fixed"] += 1

        # 去 span（保留内容）
        before = s
        s = re.sub(r"</?span[^>]*>", "", s, flags=re.IGNORECASE)
        if s != before:
            # 计数粗略按“发生过一次替换”记
            self.stats["html_tags_removed"] += 1

        # sup -> [^n] / [n]
        def _sup_to_ref(m):
            raw = m.group(1)

            # 去掉 markdown 链接结构 [xxx](url)，只留文本部分
            raw = re.sub(r"\[([^\]]*)\]\([^)]+\)", r"\1", raw)

            # 去除残留 HTML 标签
            raw = re.sub(r"</?[^>]+>", "", raw)

            # 提取数字
            num = re.search(r"\[(\d+)\]|\b(\d+)\b", raw)
            if num:
                n = num.group(1) or num.group(2)
                self.stats["sup_refs_normalized"] += 1
                if self.convert_sup_to_footnote:
                    return f"[^{n}]"
                return f"[{n}]"
            # 无法提取编号则删除 sup 块
            self.stats["sup_refs_normalized"] += 1
            return ""

        s = re.sub(
            r"<sup[^>]*>(.*?)</sup>",
            _sup_to_ref,
            s,
            flags=re.IGNORECASE | re.DOTALL,
        )

        # 删除 pandoc/calibre 生成的 index_split 锚点
        before = s
        s = re.sub(r"\(#index_split_[^)]+\)", "", s, flags=re.IGNORECASE)
        if s != before:
            self.stats["index_anchors_removed"] += 1

        return s

    # -----------------------------
    # 2) HTML 去噪（保留正文）
    # -----------------------------
    def strip_html_keep_text(self, text: str) -> str:
        s = text

        # script/style 整块删
        before = s
        s = re.sub(r"<script[^>]*>.*?</script>", "", s, flags=re.IGNORECASE | re.DOTALL)
        s = re.sub(r"<style[^>]*>.*?</style>", "", s, flags=re.IGNORECASE | re.DOTALL)

        # 其他标签删掉，仅保留文本
        s2 = re.sub(r"</?[^>]+>", "", s)
        if s2 != s:
            self.stats["html_tags_removed"] += 1
        s = s2

        if s != before:
            pass

        return self._collapse_spaces(s)

    # -----------------------------
    # 3) 断行修复（pandoc增强版）
    # -----------------------------
    def fix_line_breaks_pandoc(self, text: str) -> str:
        lines = text.split("\n")
        out: List[str] = []
        i = 0

        hard_break_pat = re.compile(
            r"^\s*(#{1,6}\s|[-*+]\s|\d+\.\s|```|>\s|---+$|\|)"
        )

        # 当前行末尾若是明确句终，不建议与下一行合并
        end_punct = re.compile(r"[。！？!?；;：:]$|[.?!]$|[”\"’']$")

        # 下一行若以下模式开头，通常是续句
        likely_cont = re.compile(r"^[a-z0-9（(“\"'《【\u4e00-\u9fff]")

        while i < len(lines):
            cur = lines[i].rstrip()

            if not cur.strip():
                out.append(cur)
                i += 1
                continue

            if i + 1 < len(lines):
                nxt = lines[i + 1].lstrip()

                cur_not_end = not end_punct.search(cur)
                nxt_not_start = not hard_break_pat.match(nxt)
                nxt_is_cont = bool(likely_cont.match(nxt))

                # 避免把 markdown 标题行并掉
                cur_is_heading = bool(re.match(r"^\s*#{1,6}\s", cur))

                if (not cur_is_heading) and cur_not_end and nxt_not_start and nxt_is_cont:
                    out.append(cur + " " + nxt)
                    self.stats["lines_merged"] += 1
                    i += 2
                    continue

            out.append(cur)
            i += 1

        s = "\n".join(out)
        return self._collapse_spaces(s)

    # -----------------------------
    # 4) 可选：删除疑似“书名/页眉页脚”独立行
    # -----------------------------
    def remove_book_titles_from_content(self, text: str) -> str:
        if not self.remove_title_lines:
            return text

        out = []
        for line in text.split("\n"):
            stripped = line.strip()
            if not stripped:
                out.append(line)
                continue

            matched = any(re.search(pat, stripped) for pat in self.book_title_patterns)
            if matched:
                self.stats["book_title_lines_removed"] += 1
                continue
            out.append(line)

        return "\n".join(out)

    # -----------------------------
    # 5) 单块清洗流水线
    # -----------------------------
    def clean_block_text(self, text: str) -> str:
        s = self._normalize_unicode(text)
        s = self.normalize_pandoc_refs(s)        # 先做引用归一化
        s = self.strip_html_keep_text(s)         # 去标签
        s = self.fix_line_breaks_pandoc(s)       # 修断行
        s = self.remove_book_titles_from_content(s)
        s = self._collapse_spaces(s)
        return s

    # -----------------------------
    # 6) 按行解析：尽量保留 Markdown 结构
    # -----------------------------
    def parse_markdown(self, raw_text: str) -> str:
        """
        规则：
        - 标题行（#）原样保留（仅做轻量规范）
        - 代码块 ``` 原样保留，不做正文清洗
        - 普通段落累计成 block 后统一 clean_block_text
        """
        lines = raw_text.split("\n")
        out: List[str] = []

        in_code_block = False
        buf: List[str] = []

        def flush_buf():
            nonlocal buf
            if not buf:
                return
            block = "\n".join(buf).strip("\n")
            if block.strip():
                cleaned = self.clean_block_text(block)
                if cleaned:
                    out.append(cleaned)
            buf = []

        for line in lines:
            # code fence
            if line.strip().startswith("```"):
                if in_code_block:
                    # 结束代码块
                    out.append(line.rstrip())
                    in_code_block = False
                else:
                    # 开始代码块：先冲刷正文 buffer
                    flush_buf()
                    out.append(line.rstrip())
                    in_code_block = True
                continue

            if in_code_block:
                out.append(line.rstrip("\n"))
                continue

            # 标题行：先冲刷 buffer，再写标题
            if re.match(r"^\s*#{1,6}\s", line):
                flush_buf()
                out.append(self._collapse_spaces(line))
                continue

            # 空行：结束当前段
            if not line.strip():
                flush_buf()
                out.append("")
                continue

            # 普通正文累积
            buf.append(line.rstrip())

        flush_buf()

        # 收尾：压缩多空行
        result = "\n".join(out)
        result = re.sub(r"\n{3,}", "\n\n", result).strip() + "\n"
        return result

    # -----------------------------
    # 7) 文件入口
    # -----------------------------
    def clean_file(self) -> Tuple[Path, Dict[str, int]]:
        if not self.input_path.exists():
            raise FileNotFoundError(f"输入文件不存在: {self.input_path}")

        raw = self.input_path.read_text(encoding="utf-8", errors="ignore")
        cleaned = self.parse_markdown(raw)
        self.output_path.write_text(cleaned, encoding="utf-8")

        return self.output_path, self.stats



In [ ]:

from pathlib import Path
import traceback

# 这里假设 MarkdownCleaner 类已经在前一个 cell 定义并执行过
# 如果没有，请先运行定义类的那个 cell

def batch_clean_markdown(
    input_dir="output_markdown",
    output_dir="cleaned_md",
    remove_title_lines=True,
    convert_sup_to_footnote=True,
):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)

    print(f"输入目录: {input_dir}")
    print(f"输出目录: {output_dir}")

    if not input_dir.exists():
        print(f"✗ 输入目录不存在: {input_dir}")
        return

    if not output_dir.exists():
        output_dir.mkdir(parents=True, exist_ok=True)
        print(f"创建输出目录: {output_dir}")

    md_files = sorted(list(input_dir.glob("*.md")))
    print(f"\n找到 {len(md_files)} 个Markdown文件\n")

    success, failed = 0, 0
    fail_list = []

    for md_file in md_files:
        print("=" * 60)
        print(f"处理: {md_file.name}")
        print("=" * 60)

        try:
            out_file = output_dir / md_file.name

            # 关键：每个文件都实例化 cleaner
            cleaner = MarkdownCleaner(
                input_path=str(md_file),
                output_path=str(out_file),
                remove_title_lines=remove_title_lines,
                convert_sup_to_footnote=convert_sup_to_footnote,
            )

            out_path, stats = cleaner.clean_file()

            print(f"✓ 完成: {out_path.name}")
            print(
                "  统计: "
                f"sup_refs={stats.get('sup_refs_normalized',0)}, "
                f"anchors={stats.get('index_anchors_removed',0)}, "
                f"html={stats.get('html_tags_removed',0)}, "
                f"merged={stats.get('lines_merged',0)}"
            )
            success += 1

        except Exception as e:
            failed += 1
            fail_list.append((md_file.name, str(e)))
            print(f"✗ 处理失败: {e}")
            # 如需详细栈可打开下一行
            # traceback.print_exc()

        print()

    print("\n" + "=" * 60)
    print("批处理结束")
    print("=" * 60)
    print(f"成功: {success}")
    print(f"失败: {failed}")

    if fail_list:
        print("\n失败文件列表:")
        for i, (fname, err) in enumerate(fail_list, 1):
            print(f"{i}. {fname} -> {err}")

# 运行
batch_clean_markdown(
    input_dir="output_markdown",
    output_dir="cleaned_md",
    remove_title_lines=True,
    convert_sup_to_footnote=True,  # True => [^7], False => [7]
)

输入目录: output_markdown
输出目录: cleaned_md

找到 5 个Markdown文件

处理: 中国巨债：经济奇迹的根源与未来 (刘海影) (Z-Library).md
✓ 完成: 中国巨债：经济奇迹的根源与未来 (刘海影) (Z-Library).md
  统计: sup_refs=0, anchors=0, html=0, merged=3124

处理: 人地之间：中国增长模式下的城乡土地改革 (陶然) (Z-Library).md
✓ 完成: 人地之间：中国增长模式下的城乡土地改革 (陶然) (Z-Library).md
  统计: sup_refs=0, anchors=0, html=0, merged=5034

处理: 当代中国政府与政治 (景跃进 陈明明 肖滨) (Z-Library).md
✓ 完成: 当代中国政府与政治 (景跃进 陈明明 肖滨) (Z-Library).md
  统计: sup_refs=0, anchors=0, html=0, merged=5031

处理: 当代中国的国家与社会关系 (周雪光 主编) (Z-Library).md
✓ 完成: 当代中国的国家与社会关系 (周雪光 主编) (Z-Library).md
  统计: sup_refs=0, anchors=0, html=5, merged=3119

处理: 新中国财政金融制度变迁事件解读 (陈雨露 郭庆旺) (Z-Library).md
✓ 完成: 新中国财政金融制度变迁事件解读 (陈雨露 郭庆旺) (Z-Library).md
  统计: sup_refs=0, anchors=0, html=0, merged=9471


批处理结束
成功: 5
失败: 0
